In [1]:
#trial creating ngram tree


In [89]:
library(dplyr)
library(tidytext)
library(data.tree)
library(stringr)



Attaching package: ‘stringr’

The following object is masked _by_ ‘.GlobalEnv’:

    words



In [3]:
enUsTwitter10kSubset <- readRDS(file='../data/subsets/enUsTwitter10kSubset.rds')

In [4]:
head(enUsTwitter10kSubset)

[[1]]
[1] "These late night walks are still religion to me. Thank goodness for these."

[[2]]
[1] "Enter to win an Orioles vintage cap by email or tweet the answer to: Which 2 O's pitchers hold the record for the most opening day starts?"

[[3]]
[1] "A Monthly payment plan is now available for those wishing to enroll in Hawaii's #1 RE Investor training program - the Millionaire Mentor..."

[[4]]
[1] "Game time!"

[[5]]
[1] "I feared that this was bound to happen Sugarland almost died 2 who should they sue The State of Indiana? whynot"

[[6]]
[1] "ADDY Awards tonight...plus giving scholarships to 5 deserving students...and something from our young advertising professionals"

In [94]:
#copy/paste processing from
# https://www.tidytextmining.com/tidytext.html

sosToken <- 'start-of-sequence' #'root' token, representing start of a sequence
eolToken <- 'eol' #token to capture end of line

text <- unlist(enUsTwitter10kSubset)

text_df <- tibble(line = 1:length(text), text = text)
text_df <- mutate(text_df, text = paste(text, eolToken))

words <- text_df %>%
  unnest_tokens(word, text) %>%
  select(word)

words$index <- 1:nrow(words)

head(text_df)

head(words, 20)
tail(words)

line,text
1,These late night walks are still religion to me. Thank goodness for these. eol
2,Enter to win an Orioles vintage cap by email or tweet the answer to: Which 2 O's pitchers hold the record for the most opening day starts? eol
3,A Monthly payment plan is now available for those wishing to enroll in Hawaii's #1 RE Investor training program - the Millionaire Mentor... eol
4,Game time! eol
5,I feared that this was bound to happen Sugarland almost died 2 who should they sue The State of Indiana? whynot eol
6,ADDY Awards tonight...plus giving scholarships to 5 deserving students...and something from our young advertising professionals eol


word,index
these,1
late,2
night,3
walks,4
are,5
still,6
religion,7
to,8
me,9
thank,10


word,index
who,137269
research,137270
and,137271
download,137272
kidzui,137273
eol,137274


In [6]:
# expand node:
# for word in unique next words:
#   find occurrences, create node

In [132]:
rootNode <- Node$new(sosToken)
rootNode$occurrences <- 0:(nrow(words)-1)
rootNode$nOccurrences <- length(rootNode$occurrences)

expandNode <- function(node) {
    nextWordIndices <- node$occurrences + 1
    nextWords <- words[nextWordIndices,]
    
    newNodes <- tibble(nodePath=character(), nOccurrences=integer())
    for (uniqueWord in unique(nextWords$word)){
        newNode <- node$AddChild(uniqueWord)
        newNode$occurrences <- filter(nextWords, word==uniqueWord)$index
        newNode$nOccurrences <- length(newNode$occurrences)
        nodePath <- newNode$pathString
        nodePath <- str_replace(nodePath, sosToken, '.')
        newNodes <- bind_rows(
            tibble(nodePath=nodePath, nOccurrences=newNode$nOccurrences),
            newNodes
        )
    }
    if (nrow(newNodes) > 0)
    {
        print(newNodes)
        Sort(node, 'nOccurrences', decreasing = T)
    }
    newNodes
}

In [127]:
frontier <-tibble(nodePath='.', nOccurrences=rootNode$nOccurrences)
frontier

nodePath,nOccurrences
.,137274


In [128]:
#expand next node:
# take the next most common sequence in the frontier
frontier <- arrange(frontier, desc(nOccurrences))
pathToExpand <- frontier$nodePath[1]
cat(paste('Expanding:', pathToExpand))
nodeToExpand <- Navigate(rootNode, pathToExpand)

# remove it from the frontier
frontier <- frontier[-1,]

# expand it
newNodes <- expandNode(nodeToExpand)

# add new nodes to frontier
frontier <- bind_rows(frontier, newNodes)
head(frontier)

Expanding: .

Warning message in CheckNameReservedWord(name, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'position' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'position2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'children' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'children2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'level' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'level2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'fields' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'fields2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'path' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'path2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'leaves' is

# A tibble: 1 x 2
  nodePath nOccurrences
  <chr>           <int>
1 ./geared            1


nodePath,nOccurrences
./geared,1


In [134]:
print(rootNode, 'nOccurrences', limit = 20)

          levelName nOccurrences
1 start-of-sequence       137274


In [130]:
expandNode(rootNode$the)

Warning message in CheckNameReservedWord(name, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'path' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'path2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'level' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'level2' instead.”

# A tibble: 1 x 2
  nodePath      nOccurrences
  <chr>                <int>
1 ./the/coconut            1


nodePath,nOccurrences
./the/coconut,1


In [131]:
expandNode(rootNode$'the'$'best')

# A tibble: 1 x 2
  nodePath           nOccurrences
  <chr>                     <int>
1 ./the/best/stickum            1


nodePath,nOccurrences
./the/best/stickum,1


In [125]:
print(rootNode, 'nOccurrences', limit = 20)

          levelName nOccurrences
1 start-of-sequence       137274


In [133]:
print(rootNode$'the', limit = 20)

NULL
